### Multi Processing

#### Word In N Gram And Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# This data is constant
word_num = 300
df_word = pd.read_csv("/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/NLTK/Result/Word/Merge_Word/Word_Merge.csv") # word merge dataframe
df_word = df_word.head(word_num)
df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
295,mutlu,254555
296,ondan,252864
297,eminim,252546
298,insan,252056


In [4]:
word_list = df_word.iloc[:,0].values.tolist()

In [5]:
# This data is variable
data_kind = "fivegram" # twogram, fourgram
df = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/Pyspark/Result/N Gram/Merge/{data_kind.capitalize()}_Merge.csv") # sentence, n gram dataframe
df

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [6]:
d_list  = df.iloc[:,0].values.tolist()

In [7]:
resultlist = []

In [8]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [9]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,bu da ne demek şimdi,4791
4,hayır hayır hayır hayır hayır,4380
...,...,...
24730,oh evet bunun için harika,10
24731,ya da pek çok yerde,10
24732,bir yanlış ve bir de,10
24734,karşı çok iyi bir yere,10


In [10]:
#df_merge_result.to_csv(f"{data_kind.capitalize()}_With_{word_num}_Word.csv", index=False)

#### N Gram In N Gram

In [11]:
word_num = 300
data_kind1 = "fourgram"
df_first = pd.read_csv(f"{data_kind1.capitalize()}_With_{word_num}_Word.csv")
#df_first = df_first.head(1000)
df_first

,fourgram,frequency
0,bir şey var mı,41773
1,bir sorun mu var,21423
2,ben de seni seviyorum,17338
3,bu da ne demek,16884
4,gereken bir şey var,15683
...,...,...
995,hem de çok kötü,444
996,bu kadar da değil,444
997,ve sen bunu biliyorsun,444
998,da da da da,444


In [12]:
df_first_list = df_first.iloc[:,0].to_list()

In [13]:
data_kind2 = "fivegram"
df_second = pd.read_csv(f"{data_kind2.capitalize()}_With_{word_num}_Word.csv")
#df_second = df_second.head(1000)
df_second 

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,bu da ne demek şimdi,4791
4,hayır hayır hayır hayır hayır,4380
...,...,...
995,her şey çok daha güzel,106
996,evet mi yoksa hayır mı,106
997,sana bir şey oldu mu,106
998,bu ne demek biliyorsun değil,106


In [14]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_df = manager.list()

def wordgroup_in_wordgroup(df_first_list):
    df_wordgroup_result = pd.DataFrame()
    df_result = df_second[df_second.iloc[:,0].str.contains(fr"(^|\s){df_first_list}(\s|$)")]
    df_result.insert(0,f"{data_kind1}",df_first_list)
    df_wordgroup_result = pd.concat([df_wordgroup_result,df_result], axis=0)
    df_wordgroup_result.reset_index(drop=True, inplace=True)
    result_df.append(df_wordgroup_result)    
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(wordgroup_in_wordgroup, df_first_list) # df_first_list liste

In [15]:
multi_df_list = list(result_df)
df_concat = pd.DataFrame()
for i in multi_df_list:
    df_concat = pd.concat([df_concat,i],axis=0)
df_concat.sort_values(by="frequency",ascending=False, inplace=True)
df_concat.reset_index(drop=True, inplace=True)
df_concat

,fourgram,fivegram,frequency
0,başka bir şey var,başka bir şey var mı,14104
1,bir şey var mı,başka bir şey var mı,14104
2,da ne demek oluyor,bu da ne demek oluyor,10205
3,bu da ne demek,bu da ne demek oluyor,10205
4,kadar da kötü değil,o kadar da kötü değil,7012
...,...,...,...
951,için bu kadar önemli,için bu kadar önemli olduğunu,106
952,her şey çok daha,her şey çok daha güzel,106
953,bir şey oldu mu,sana bir şey oldu mu,106
954,çok şey var ki,öyle çok şey var ki,106


In [16]:
#df_concat.to_csv(f"{data_kind1.capitalize()}_In_{data_kind2.capitalize()}_With_{word_num}_Word.csv", index=False)

#### N Gram In Sentence

In [17]:
data_kind = "sentence"
df_n_sent = pd.read_csv(f"{data_kind.capitalize()}_With_{word_num}_Word.csv")
df_n_sent

,sentence,frequency,code
0,evet,1948596,G1-L4-F1948596-0
1,hayır,1250401,G1-L5-F1250401-2
2,tamam,882921,G1-L5-F882921-3
3,ne,753105,G1-L2-F753105-4
4,teşekkürler,343527,G1-L11-F343527-5
...,...,...,...
69797,bize tek gereken bu,6,G4-L19-F6-2854680
69798,ve bu iyi bir şey değil mi,6,G7-L26-F6-2854591
69799,bize biraz zaman ver olur mu,6,G6-L28-F6-2854551
69800,bir gün senin de olacak,6,G5-L23-F6-2854424


In [18]:
data_kind1 = "twogram"
data_kind2 = "threegram"
df_two_in_three = pd.read_csv(f"{data_kind1.capitalize()}_In_{data_kind2.capitalize()}_With_{word_num}_Word.csv")
df_two_in_three

,twogram,threegram,frequency
0,şey yok,bir şey yok,113165
1,bir şey,bir şey yok,113165
2,şey var,bir şey var,110455
3,bir şey,bir şey var,110455
4,bu da,bu da ne,89463
...,...,...,...
1588,için her,için her zaman,2405
1589,güzel bir,daha güzel bir,2404
1590,belki de,belki de bunu,2402
1591,de bunu,belki de bunu,2402


In [19]:
data_kind1 = "threegram"
data_kind2 = "fourgram"
df_three_in_four = pd.read_csv(f"{data_kind1.capitalize()}_In_{data_kind2.capitalize()}_With_{word_num}_Word.csv")
df_three_in_four

,threegram,fourgram,frequency
0,bir şey var,bir şey var mı,41773
1,şey var mı,bir şey var mı,41773
2,bir sorun mu,bir sorun mu var,21423
3,sorun mu var,bir sorun mu var,21423
4,de seni seviyorum,ben de seni seviyorum,17338
...,...,...,...
1199,bu iyi bir,evet bu iyi bir,445
1200,hem de çok,hem de çok kötü,444
1201,bir daha asla,bir daha asla böyle,444
1202,kadar da değil,bu kadar da değil,444


In [20]:
data_kind1 = "fourgram"
data_kind2 = "fivegram"
df_four_in_five = pd.read_csv(f"{data_kind1.capitalize()}_In_{data_kind2.capitalize()}_With_{word_num}_Word.csv")
df_four_in_five

,fourgram,fivegram,frequency
0,bir şey var mı,başka bir şey var mı,14104
1,başka bir şey var,başka bir şey var mı,14104
2,da ne demek oluyor,bu da ne demek oluyor,10205
3,bu da ne demek,bu da ne demek oluyor,10205
4,o kadar da kötü,o kadar da kötü değil,7012
...,...,...,...
951,o kadar kötü değil,o kadar kötü değil mi,106
952,çok şey var ki,öyle çok şey var ki,106
953,her şey çok daha,her şey çok daha güzel,106
954,bir şey oldu mu,sana bir şey oldu mu,106


##### N Gram Merge

In [21]:
df_two_three_four_merge = pd.merge(df_two_in_three, df_three_in_four, how="inner", on="threegram")
df_two_three_four_merge.drop_duplicates(inplace=True)
df_two_three_four_merge

,twogram,threegram,frequency_x,fourgram,frequency_y
0,şey yok,bir şey yok,113165,diye bir şey yok,6532
1,şey yok,bir şey yok,113165,başka bir şey yok,5895
2,şey yok,bir şey yok,113165,bir şey yok mu,5016
3,şey yok,bir şey yok,113165,hiç bir şey yok,3181
4,şey yok,bir şey yok,113165,öyle bir şey yok,3165
...,...,...,...,...,...
2147,benim için,benim için büyük,2419,bu benim için büyük,492
2148,o da,o da senin,2413,o da senin gibi,805
2149,da öyle,ya da öyle,2408,ya da öyle bir,1507
2150,ya da,ya da öyle,2408,ya da öyle bir,1507


In [22]:
df_two_three_four_five_merge = pd.merge(df_two_three_four_merge,df_four_in_five, how="inner", on="fourgram")
df_two_three_four_five_merge.drop_duplicates(inplace=True)
df_two_three_four_five_merge

,twogram,threegram,frequency_x,fourgram,frequency_y,fivegram,frequency
0,şey yok,bir şey yok,113165,diye bir şey yok,6532,biz diye bir şey yok,1356
1,şey yok,bir şey yok,113165,diye bir şey yok,6532,diye bir şey yok mu,134
2,şey yok,bir şey yok,113165,diye bir şey yok,6532,yarın diye bir şey yok,125
3,şey yok,bir şey yok,113165,diye bir şey yok,6532,olacak diye bir şey yok,116
4,bir şey,bir şey yok,113165,diye bir şey yok,6532,biz diye bir şey yok,1356
...,...,...,...,...,...,...,...
2573,de iyi,hiç de iyi,2505,hiç de iyi değil,851,bu hiç de iyi değil,684
2574,hiç de,hiç de iyi,2505,hiç de iyi değil,851,bu hiç de iyi değil,684
2575,git git,git git git,2421,git git git git,464,git git git git git,216
2576,da öyle,ya da öyle,2408,ya da öyle bir,1507,ya da öyle bir şey,1396


In [23]:
df_two_three_four_five_ngram = df_two_three_four_five_merge[["twogram", "threegram", "fourgram", "fivegram"]]
df_two_three_four_five_ngram

,twogram,threegram,fourgram,fivegram
0,şey yok,bir şey yok,diye bir şey yok,biz diye bir şey yok
1,şey yok,bir şey yok,diye bir şey yok,diye bir şey yok mu
2,şey yok,bir şey yok,diye bir şey yok,yarın diye bir şey yok
3,şey yok,bir şey yok,diye bir şey yok,olacak diye bir şey yok
4,bir şey,bir şey yok,diye bir şey yok,biz diye bir şey yok
...,...,...,...,...
2573,de iyi,hiç de iyi,hiç de iyi değil,bu hiç de iyi değil
2574,hiç de,hiç de iyi,hiç de iyi değil,bu hiç de iyi değil
2575,git git,git git git,git git git git,git git git git git
2576,da öyle,ya da öyle,ya da öyle bir,ya da öyle bir şey


In [24]:
df_two_three_four_five_ngram.twogram.nunique()

278

In [25]:
df_two_three_four_five_ngram.threegram.nunique()

362

In [26]:
df_two_three_four_five_ngram.fourgram.nunique()

469

In [27]:
df_two_three_four_five_ngram.fivegram.nunique()

651

In [28]:
n_gram_list = df_two_three_four_five_ngram.iloc[:,3].to_list() # last n gram values

In [29]:
#df_n_sent.iloc[:,0]

In [30]:
result_df = []

In [31]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_df = manager.list()

def wordgroup_in_wordgroup(n_gram_list):
    df_wordgroup_result = pd.DataFrame()
    df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
    df_result.insert(0,f"all_ngram",n_gram_list)
    df_wordgroup_result = pd.concat([df_wordgroup_result,df_result], axis=0)
    df_wordgroup_result.reset_index(drop=True, inplace=True)
    result_df.append(df_wordgroup_result)    
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(wordgroup_in_wordgroup, n_gram_list) # string_word liste

In [32]:
multi_df_sent_list = list(result_df)
df_sent_result = pd.DataFrame()
for i in multi_df_sent_list:
    df_sent_result = pd.concat([df_sent_result,i],axis=0)
df_sent_result.drop_duplicates(inplace=True)
df_sent_result.sort_values(by="frequency",ascending=False, inplace=True)
df_sent_result.reset_index(drop=True, inplace=True)
df_sent_result

,all_ngram,sentence,frequency,code
0,bu da ne demek oluyor,bu da ne demek oluyor,4633,G5-L21-F4633-1154
1,başka bir şey var mı,başka bir şey var mı,4264,G5-L20-F4264-1256
2,bu da ne demek şimdi,bu da ne demek şimdi,2360,G5-L20-F2360-2326
3,o kadar da kötü değil,o kadar da kötü değil,2311,G5-L21-F2311-2375
4,bunun ne olduğunu biliyor musun,bunun ne olduğunu biliyor musun,1872,G5-L31-F1872-2924
...,...,...,...,...
1317,iyi bir şey değil mi,ve bu iyi bir şey değil mi,6,G7-L26-F6-2854591
1318,o kadar da kötü değil,o kadar da kötü değil bence,6,G6-L27-F6-2871513
1319,bu iyi bir şey değil,ve bu iyi bir şey değil mi,6,G7-L26-F6-2854591
1320,o kadar da kötü değil,bu adam o kadar da kötü değil,6,G7-L29-F6-2818333


In [33]:
#df_sent_result.to_excel("Two_Three_Four_Five_Ngram_In_Sentence.xlsx", index=False, encoding="utf-8")